In [18]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import WebBaseLoader
import bs4


loader = WebBaseLoader(
    web_paths= ("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    )
)


docs = loader.load()


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)




splits = text_splitter.split_documents(docs)


In [20]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


vectorstore = Chroma.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={"k" : 1, "fetch_k" : 4}
)


In [21]:
bm25_retriever = BM25Retriever.from_documents(splits)




bm25_retriever.k = 2




ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                weights=[0.2, 0.8])




ensemble_retriever.invoke("오늘의 증시")


[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='이데일리\n\n18분전\n\n\n\n\n\n\n\n\n[아침밥] 하나증권 "롯데관광개발, 2분기 사상 최대 실적… 목표가 쭉"'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="에이피알(278470)이 1분기에 이어 2분기에도 신기록을 세우며 '연 매출 1조 원' 목표에 성큼 다가섰다. 에이피알은 2분기 경영실적을 잠정 집계한 결과 연결 기준 매출액이 3277억 원으로 전년 동기 대비 11\n\n\n뉴스1\n\n27분전"),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='한국경제\n\n14분전\n\n\n\n\n\n\n\n\n‘마스가’ 수혜 기대감↑…미래에셋, ‘TIGER 조선TOP10 ETF’ 순자산 4000억원 돌파 [투자360]')]

서로 다른 질문을 물어보고, 지능적으로 결합 후 그중 순위를 만들어 출력

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


In [16]:
template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)




from langchain.load import dumps, loads

In [23]:
def reciprocal_rank_fusion(results, k=60, top_n=2):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(docs)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
   
    reranked_results = [ (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x : x[1], reverse=True)
    ]
    return reranked_results[:top_n]

In [24]:
chain = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion
from langchain_core.runnables import RunnablePassthrough


template = """다음 맥락을 바탕으로 질문에 답변할 것
{context}


질문: {question}
"""


prompt = ChatPromptTemplate.from_template(template)


In [26]:
final_chain = (
    {
        "context" : chain,
        "question" : RunnablePassthrough()
    }
    | prompt | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser())

OpenAI 오디오 스피치

In [27]:
rt = final_chain.invoke("오늘의 증시")




from openai import OpenAI
client = OpenAI()
response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=rt
    )


response.stream_to_file("./output.mp3")


/tmp/ipykernel_13568/814652417.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  reranked_results = [ (loads(doc), score)
/tmp/ipykernel_13568/1925078889.py:15: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("./output.mp3")


###  pdf 에서 텍스트 추출 후 Rag에 이용하기

In [9]:
from langchain_text_splitters import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

In [17]:
def extract_pdf_elements(path, fname):
    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,  # PDF에서 이미지를 추출
        infer_table_structure=True,  # 테이블 구조를 추론
        chunking_strategy="by_title",  # 타이틀을 기준으로 텍스트를 블록으로 분할
        max_characters=4000,  # 최대 4000자로 텍스트 블록을 제한
        new_after_n_chars=3800,  # 3800자 이후에 새로운 블록 생성
        combine_text_under_n_chars=2000,  # 2000자 이하의 텍스트는 결합
        image_output_dir_path=path,  # 이미지가 저장될 경로 설정
        languages=['kor']
        # image_output_dir_path=os.path.join(os.getcwd(),"figures"),
    )


In [18]:
raw_data = extract_pdf_elements("./", "RRF.pdf")



In [21]:
def categorize_elements(raw_pdf_elements):
    """
    PDF에서 추출한 요소들을 테이블과 텍스트로 분류합니다.
    raw_pdf_elements: unstructured.documents.elements 리스트
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))  # 테이블 요소를 저장
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))  # 텍스트 요소를 저장
    return texts, tables




texts, tables = categorize_elements(raw_data)


In [22]:
texts

['띠 여러 개의 서로 다 는 ㄴㄴ [라간 ( 목 6000『0081 『<801<『46100) 하여 하 나 의 니다. 퓨 전 (『48100) 알고리즘 중 으 브 만드는 알고리즘 매긴 점 수 (6006) 가 아 있습니다.\n\n00 101 0 어 , 81125 를 들 근 크 여 을 때 사 용 됩니다. 결 과 를 만들고\n\n따 간의 핵심 원리: 역 순 위 (60000 『8010\n\n심은 역 순 위 (\\6 어 0『0 ㅇ ㅇ 801 800) 개 념 에 있습니다. 의 따 간의 이\n\n1/7( 마 에 있다면, 아 가 에서 번째 41 시 연 역 + 0) 가 됩니다.\n\n서 의 순위 ((3010) 므 느 :\n\n} 수 (보통 60 사용) 6: 순 위 가 너무 높아서 점 수 가 과 도 하게 커지\n\n과 같이 점 수 가 매 겨 집니다. 이는 상 전 런 … 1/62 느 ㄴ 1/61 점 , 2 위 느 ㄴ 니다. 1 위 } 번 힌\n\n(5[60-0+「-5[60) <! ㅁ 0 내 0 깨 [ [ 따라\n\n니다. 101 『 쥐 [ 1 / (< + 603) 를 계\n\n0 \\ ㅁ 하 라 85 이 이 ~ 마 해당 으로 나타나는 문 서 가 있다면\n\n마란 예제\n\n두 개의 검색 시 스 템 (&, 6) 이 3 개 의 문 서 (0001, 0062, 0063) 에 대해 다 음 과 같 해 봅시다. (6=60 사용) [10 다 그 0\n\n검색 시스템 4 의 결과\n\n0001\n\n0003\n\n검색 시스템 6 의 결과\n\n0001\n\n스템 & 에 서 1 위 : 1 / (1 + 60) =0.0164\n\n스템 8 에 서 2 위 : 1 / (2 + 60) =0.0161\n\n0002:\n\n스템 & 에 서 순위 없음: 0 점\n\n스템 8 에 서 1 위 : 1 / (1 + 60) =0.0164\n\n0003:\n\n스템 & 에 서 2 위 : 1 / (2 + 60) =0.0161\n\n스템 6 에 서 순위 없음: 0 점\n\n2. 점수 합산\n\n0001 종점: 0.0164 + 0

In [23]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=200
)


In [24]:
joined_texts = "".join(texts)


In [26]:
text_token = text_splitter.split_text(joined_texts)


### 로드한 문서 데이터를 요약하기

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
prompt_text = """
당신은 표와 텍스트를 요약해 검색에 활용할 수 있도록 도와주는 도우미 입니다.
이 요약본들은 임베딩되어 원본 ㄴ텍스트나 표 요소를 검색하는데 사용될 것입니다.
주어진 표나 텍스트의 내용을 검색에 최적화된 간결한 요약으로 작성하세요.
요약할 표 또는 텍스트 : {element}
"""

In [29]:
prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)

In [30]:
summaries_chain = (
    {
        'element' : lambda x : x
    }
    | prompt | model | StrOutputParser()


)

text_summaries = summaries_chain.batch(texts, {'max_concurrency' : 5})


In [31]:
text_summaries

['요약:  \n이 문서는 여러 검색 시스템의 결과를 통합하는 알고리즘인 Reciprocal Rank Fusion (RRF)에 대해 설명한다. RRF는 각 문서의 순위에 따라 1 / (k + rank) 방식으로 점수를 부여하고, 여러 시스템에서의 점수를 합산하여 최종 순위를 결정한다. 이 방식은 단순하고 직관적이며, 점수 정규화가 필요 없고, 특정 시스템의 성능 저하에도 견고하게 작동한다. 하이브리드 검색 시스템에서 효과적으로 사용된다.']

## 데이터셋을 가져오기

In [14]:
# from datasets import load_dataset
# 허깅페이스 fasionpedia 데이터 사용

In [3]:
import zipfile
with zipfile.ZipFile("./fashion_dataset.zip", 'r') as f:
    f.extractall("./fashion/")

In [6]:
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
import chromadb


chroma_client = chromadb.PersistentClient("./fvector")

In [7]:
image_loader = ImageLoader()

In [11]:
CLIP = OpenCLIPEmbeddingFunction()

In [12]:
image_vdb = chroma_client.get_or_create_collection(
        name="image", embedding_function=CLIP, data_loader=image_loader)

In [15]:
import os
ids = []
uris =[]
roots = "./fashion/fashion_dataset/"
for idx, filename in enumerate(sorted(os.listdir(roots))):
    if filename.endswith(".png"):
        uris.append(roots+filename)
        ids.append(str(idx))

In [18]:
image_vdb.add(ids=ids, uris=uris)

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def translate(text):
    model  = ChatOpenAI(model='gpt-4.1', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', "You are a translator. Translate the following text to English"),
            ('user', "{text}")
        ]
    )


    chain = prompt | model | StrOutputParser()
    return chain.invoke({'text' : text})




query = translate("검은색 티셔츠와 청바지 추천")

In [26]:
def query_db(image_vdb, query, results=2):
    # 주어진 쿼리를 실행하고, 상위 결과 반환
    return image_vdb.query(
        query_texts=[query],
        n_results=results,
        include=['uris', 'distances'])

In [27]:
tmp = image_vdb.query(
    query_texts=[query],
    n_results=2,
    include=['uris', 'distances']
)


In [ ]:
def setup_chain():
    gpt4 = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    parser = StrOutputParser()
    image_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful fashion and styling assistant. Answer the user's question using the given image context with direct references to parts of the images provided. Maintain a more conversational tone, don't make too many lists. Use markdown formatting for highlights, emphasis, and structure."),
            ("user", [
                {"type": "text", "text": "What are some ideas for styling {user_query}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_2}"},
            ]),
        ])


    return image_prompt | gpt4 | parser


In [30]:
def format_prompt_inputs(data, user_query):
    inputs = {}


    inputs['user_queyr'] = user_query


    image_path_1 = data['uris'][0][0]
    image_path_2 = data['uris'][0][1]


    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')


    with open(image_path_2, 'rb') as image_file:
        image_data_2 = image_file.read()
    inputs['image_data_2'] = base64.b64encode(image_data_2).decode('utf-8')


    return inputs


In [38]:
ko_query = "바지하고 티셔츠를 파란색으로 추천"
en_query = translate(ko_query)

results = query_db(image_vdb, en_query)
prompt_input = format_prompt_inputs(results, en_query)
response_en = vision_chain.invoke(prompt_input)

AttributeError: module 'vision_chain' has no attribute 'invoke'

In [47]:
# 이미지 기반 패션 추천 파이프라인: 한글 질문→영어 번역→유사 이미지 DB 검색→이미지+영문질문 LLM 프롬프트→답변 생성→최종 한글 번역까지 엔드 투 엔드
import base64                                  # 이미지 파일 base64 인코딩용
from langchain_openai import ChatOpenAI         # LLM(챗GPT-4o 등)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def query_db(image_vdb, query, results=2):
    # 헤드라이너
    # 이미지 벡터DB(image_vdb)에 사용자(영어) 쿼리로 top-N 유사 이미지의 경로/유사도 추출
    return image_vdb.query(
        query_texts=[query],           # (리스트 형태 english query)
        n_results=results,             # 상위 N개 이미지 반환
        include=['uris', 'distances']  # 반환: 이미지 경로와 유사도 점수
    )

# 예시: 직접 DB 쿼리 가능(일반적으론 아래 함수로 래핑해서 사용)
# tmp = image_vdb.query(query_texts=[query], n_results=2, include=['uris', 'distances'])

def setup_chain():
    # LLM 프롬프트+모델+파서 구성: 자연어 '스타일링 아이디어' 생성용
    gpt4 = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    parser = StrOutputParser()
    image_prompt = ChatPromptTemplate.from_messages([
        ("system", 
         "You are a helpful fashion and styling assistant. Answer the user's question using the given image context with direct references to parts of the images provided. Maintain a more conversational tone, don't make too many lists. Use markdown formatting for highlights, emphasis, and structure."
        ),
        ("user", [
            {"type": "text", "text": "What are some ideas for styling {user_query}"},
            {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
            {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_2}"},
        ]),
    ])
    return image_prompt | gpt4 | parser

def format_prompt_inputs(data, user_query):
    # DB 쿼리 결과(data: dict)와 사용자(영문) 질문을 LLM 입력 dict로 가공
    inputs = {}
    inputs['user_query'] = user_query
    # 유사도 상위 이미지 2개의 경로 추출
    image_path_1 = data['uris'][0][0]
    image_path_2 = data['uris'][0][1]
    # 각각을 base64 인코딩해서 LLM 프롬프트에 삽입
    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')
    with open(image_path_2, 'rb') as image_file:
        image_data_2 = image_file.read()
    inputs['image_data_2'] = base64.b64encode(image_data_2).decode('utf-8')
    return inputs

def translate(text, target="english"):
    model  = ChatOpenAI(model='gpt-4.1', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', f"You are a translator. Translate the following text to {target}"),
            ('user', "{text}")
        ]
    )


    chain = prompt | model | StrOutputParser()
    return chain.invoke({'text' : text})

# 실제 전체 동작 흐름 예시
ko_query = "바지하고 티셔츠를 파란색으로 추천"                  # (1) 한글질문
en_query = translate(ko_query, 'english')                       # (2) 영어 번역
results = query_db(image_vdb, en_query)                         # (3) DB에서 top-2 유사 이미지 검색
prompt_input = format_prompt_inputs(results, en_query)          # (4) 프롬프트 입력 포맷(이미지+문장)
vision_chain = setup_chain()                                    # (5) LLM 체인 준비
response_en = vision_chain.invoke(prompt_input)                 # (6) 패션 스타일링 추천 답변(영어)
rt = translate(response_en, 'korean')                           # (7) 필요시 한글로 번역



## 논문을 검색하는 agent

In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain import hub

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_openai import ChatOpenAI
import os

In [56]:
openai = ChatOpenAI(model='gpt-4o-mini', temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")


api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
# naver news
loader = WebBaseLoader(
    "https://news.naver.com/"
)

docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())




In [ ]:
retriever = vectordb.as_retriever()
retriever_tool = create_retriever_tool(
    retriever, "naver_news_search", "네이버 뉴스정보가 저장된 벡터 DB, 당일 뉴스 검색 가능"
)
